In [ ]:
import spacy
import json

### Load models

In [ ]:
nlp_ner = spacy.load('/src/ia2/ia2/models/ner/model-best')

In [ ]:
nlp_ner.pipe_names

In [ ]:
nlp_pipe= spacy.load('/src/ia2/ia2/models/all_pipe_t2v_ner/model-best')

In [ ]:
nlp_pipe.pipe_names

### Add morphologizer and parser in NER pipeline

In [ ]:
nlp_ner.add_pipe('morphologizer', after='tok2vec', source=nlp_pipe)

In [ ]:
nlp_ner.pipe_names

In [ ]:
nlp_ner.add_pipe('parser', after='morphologizer', source=nlp_pipe)

### Add EntityRuler in NER pipeline

In [ ]:
from spacy.pipeline import EntityRuler
from ia2.pipeline.entity_ruler import fetch_ruler_patterns_by_tag

pipelines_tag = "todas"

ruler = nlp_ner.add_pipe("entity_ruler", before='ner')
ruler.add_patterns(fetch_ruler_patterns_by_tag(pipelines_tag))

In [ ]:
nlp_ner.pipe_names

In [ ]:
nlp_ner.get_pipe('ner').labels

In [ ]:
nlp_ner.get_pipe('entity_ruler').labels

### Add entity_matcher in NER pipeline

In [ ]:
from ia2.pipeline.entity_matcher import (EntityMatcher, matcher_patterns, fetch_cb_by_tag)

entity_matcher = nlp_ner.add_pipe("matcher")

In [ ]:
# from ia2.pipeline.entity_matcher import (EntityMatcher, matcher_patterns, fetch_cb_by_tag)

# entity_matcher = EntityMatcher(
#             nlp_ner,
#             matcher_patterns,
#             after_callbacks=[cb(nlp_ner) for cb in fetch_cb_by_tag(pipelines_tag)],
#         )

In [ ]:
nlp_ner.pipe_names

### Add entity_custom in NER pipeline

In [ ]:
from ia2.pipeline.entity_custom import EntityCustom

# entity_custom = EntityCustom(nlp_ner, pipelines_tag)

In [ ]:
nlp_ner.add_pipe("matcher_custom")

In [ ]:
nlp_ner.pipe_names

### Save custom model

In [ ]:
nlp_ner.to_disk('/src/ia2/ia2/models/custom_model_v2')

### Evaluate custom model

In [ ]:
! python -m spacy evaluate /src/ia2/ia2/models/custom_model_v2  /resources/datasets/docbin/test.spacy --output /resources/ouputs/metrics/custom_model/metics_v2.json --gold-preproc --code /src/ia2/ia2/pipeline/__init__.py --displacy-path /resources/ouputs/displacy/custom_model_v2 --displacy-limit 15

In [ ]:
! python -m spacy evaluate /src/ia2/ia2/models/custom_model  /resources/datasets/docbin/test.spacy --output /resources/ouputs/metrics/custom_model/metics_custom.json --gold-preproc --displacy-path /resources/ouputs/displacy/custom_model --displacy-limit 15

In [ ]:
path_data_testing = '/resources/datasets/unified/test.json'
with open(path_data_testing) as f:
    testing_data = json.load(f)

In [ ]:
text = testing_data[3][0]
doc = nlp_ner(text)

In [ ]:
doc.ents

In [ ]:
len(doc.ents)

In [ ]:
spacy.displacy.render(doc, style='ent', jupyter=True)

### Deploy custom model

In [ ]:
! python -m spacy package /src/ia2/ia2/models/custom_model_v2 /src/ia2 --code /src/ia2/ia2/pipeline/__init__.py